In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity
import re
from PIL import Image
import requests
import random

In [6]:
data_bow = pd.read_csv("merged_data_bow_final.csv")
data_bow

,ISBN,User-ID,Book-Title,Book-Rating,Prediction_bow_linreg,Prediction_bow_xgb,Prediction_bow_knn
0,39,8,Clara Callan,5,7.671816,7.529050,5.4
1,39,11676,Clara Callan,8,7.667883,7.529050,8.0
2,39,67544,Clara Callan,8,7.649050,7.529050,7.2
3,39,116866,Clara Callan,9,7.632424,7.529050,8.2
4,39,123629,Clara Callan,9,7.630144,7.529050,7.2
...,...,...,...,...,...,...,...
383837,41460,275318,Dreamsnake,10,7.579010,7.641730,9.6
383838,129765,275970,Cocktail Classics,7,7.578790,7.706649,9.4
383839,53784,276313,Flashpoints Promise and Peril in a New World,5,7.578675,7.641730,7.0
383840,48133,276463,There s a Bat in Bunk Five,7,7.578624,7.641730,7.6


In [7]:
data_tfidf = pd.read_csv("merged_data_tfidf.csv")
data_tfidf

,ISBN,User-ID,Book-Title,Book-Rating,Prediction_tfidf_linreg,Prediction_tfidf_xgb
0,39,8,Clara Callan,5,7.671816,7.527846
1,39,11676,Clara Callan,8,7.667883,7.527846
2,39,67544,Clara Callan,8,7.649050,7.527846
3,39,116866,Clara Callan,9,7.632424,7.527846
4,39,123629,Clara Callan,9,7.630144,7.527846
...,...,...,...,...,...,...
383837,41460,275318,Dreamsnake,10,7.579010,7.642559
383838,129765,275970,Cocktail Classics,7,7.578790,7.574427
383839,53784,276313,Flashpoints Promise and Peril in a New World,5,7.578675,7.583351
383840,48133,276463,There s a Bat in Bunk Five,7,7.578624,7.642559


In [8]:
data_w2v_linreg = pd.read_csv("data_w2v_linreg_final.csv")
data_w2v_linreg.rename(columns={'Predicted-Book-Rating': 'Prediction_w2v_linreg'}, inplace=True)
data_w2v_linreg

,ISBN,Book-Title,User-ID,Book-Rating,Location,Prediction_w2v_linreg
0,39,Clara Callan,8,5,"timmins, ontario, canada",7.738213
1,39,Clara Callan,11676,8,"n/a, n/a, n/a",7.992755
2,39,Clara Callan,67544,8,"toronto, ontario, canada",7.702522
3,39,Clara Callan,116866,9,"ottawa, ,",7.740668
4,39,Clara Callan,123629,9,"kingston, ontario, canada",7.689082
...,...,...,...,...,...,...
383837,41460,Dreamsnake,275318,10,"klamath falls, oregon, usa",7.840090
383838,129765,Cocktail Classics,275970,7,"pittsburgh, pennsylvania, usa",7.924090
383839,53784,Flashpoints Promise and Peril in a New World,276313,5,"livermore, california, usa",8.193919
383840,48133,There s a Bat in Bunk Five,276463,7,"providence, rhode island, usa",7.927010


In [9]:
data_w2v_xgb = pd.read_csv("data_w2v_xgb_final.csv")
data_w2v_xgb.rename(columns={'Predicted-Book-Rating': 'Prediction_w2v_xgb'}, inplace=True)
data_w2v_xgb

,ISBN,Book-Title,User-ID,Book-Rating,Location,Prediction_w2v_xgb
0,39,Clara Callan,8,5,"timmins, ontario, canada",7.645508
1,39,Clara Callan,11676,8,"n/a, n/a, n/a",7.706393
2,39,Clara Callan,67544,8,"toronto, ontario, canada",7.640519
3,39,Clara Callan,116866,9,"ottawa, ,",7.643437
4,39,Clara Callan,123629,9,"kingston, ontario, canada",7.583914
...,...,...,...,...,...,...
383837,41460,Dreamsnake,275318,10,"klamath falls, oregon, usa",7.741884
383838,129765,Cocktail Classics,275970,7,"pittsburgh, pennsylvania, usa",8.309811
383839,53784,Flashpoints Promise and Peril in a New World,276313,5,"livermore, california, usa",7.789129
383840,48133,There s a Bat in Bunk Five,276463,7,"providence, rhode island, usa",7.853196


In [10]:
data_w2v_knn = pd.read_csv("data_w2v_knn_final.csv")
data_w2v_knn.rename(columns={'Predicted-Book-Rating': 'Prediction_w2v_knn'}, inplace=True)
data_w2v_knn

,ISBN,Book-Title,User-ID,Book-Rating,Location,Prediction_w2v_knn
0,39,Clara Callan,8,5,"timmins, ontario, canada",7.2
1,39,Clara Callan,11676,8,"n/a, n/a, n/a",7.0
2,39,Clara Callan,67544,8,"toronto, ontario, canada",6.4
3,39,Clara Callan,116866,9,"ottawa, ,",7.4
4,39,Clara Callan,123629,9,"kingston, ontario, canada",7.0
...,...,...,...,...,...,...
383837,41460,Dreamsnake,275318,10,"klamath falls, oregon, usa",7.4
383838,129765,Cocktail Classics,275970,7,"pittsburgh, pennsylvania, usa",5.6
383839,53784,Flashpoints Promise and Peril in a New World,276313,5,"livermore, california, usa",6.4
383840,48133,There s a Bat in Bunk Five,276463,7,"providence, rhode island, usa",6.2


In [11]:
merged_data = pd.concat([data_bow['ISBN'], data_bow['User-ID'], data_bow['Book-Title'], data_bow['Book-Rating'], data_w2v_linreg['Location'],
                         data_bow['Prediction_bow_linreg'],
                         data_bow['Prediction_bow_xgb'],
                         data_bow['Prediction_bow_knn'],
                         data_tfidf['Prediction_tfidf_linreg'],
                         data_tfidf['Prediction_tfidf_xgb'],
                         data_w2v_linreg['Prediction_w2v_linreg'],
                         data_w2v_xgb['Prediction_w2v_xgb'],
                         data_w2v_knn['Prediction_w2v_knn']],
                        axis=1,
                        keys=['ISBN', 'User-ID', 'Book-Title', 'Book-Rating', 'Location',
                              'Prediction_bow_linreg', 'Prediction_bow_xgb', 'Prediction_bow_knn',
                              'Prediction_tfidf_linreg', 'Prediction_tfidf_xgb', 'Prediction_w2v_linreg', 'Prediction_w2v_xgb', 'Prediction_w2v_knn'])
predictions_df = merged_data[['Prediction_bow_linreg', 'Prediction_bow_xgb', 'Prediction_bow_knn',
                              'Prediction_tfidf_linreg', 'Prediction_tfidf_xgb',
                              'Prediction_w2v_linreg', 'Prediction_w2v_xgb', 'Prediction_w2v_knn']]

merged_data['Mean_Prediction'] = predictions_df.mean(axis=1)
merged_data

,ISBN,User-ID,Book-Title,Book-Rating,Location,Prediction_bow_linreg,Prediction_bow_xgb,Prediction_bow_knn,Prediction_tfidf_linreg,Prediction_tfidf_xgb,Prediction_w2v_linreg,Prediction_w2v_xgb,Prediction_w2v_knn,Mean_Prediction
0,39,8,Clara Callan,5,"timmins, ontario, canada",7.671816,7.529050,5.4,7.671816,7.527846,7.738213,7.645508,7.2,7.298031
1,39,11676,Clara Callan,8,"n/a, n/a, n/a",7.667883,7.529050,8.0,7.667883,7.527846,7.992755,7.706393,7.0,7.636476
2,39,67544,Clara Callan,8,"toronto, ontario, canada",7.649050,7.529050,7.2,7.649050,7.527846,7.702522,7.640519,6.4,7.412255
3,39,116866,Clara Callan,9,"ottawa, ,",7.632424,7.529050,8.2,7.632424,7.527846,7.740668,7.643437,7.4,7.663231
4,39,123629,Clara Callan,9,"kingston, ontario, canada",7.630144,7.529050,7.2,7.630144,7.527846,7.689082,7.583914,7.0,7.473772
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
383837,41460,275318,Dreamsnake,10,"klamath falls, oregon, usa",7.579010,7.641730,9.6,7.579010,7.642559,7.840090,7.741884,7.4,7.878035
383838,129765,275970,Cocktail Classics,7,"pittsburgh, pennsylvania, usa",7.578790,7.706649,9.4,7.578790,7.574427,7.924090,8.309811,5.6,7.709070
383839,53784,276313,Flashpoints Promise and Peril in a New World,5,"livermore, california, usa",7.578675,7.641730,7.0,7.578675,7.583351,8.193919,7.789129,6.4,7.470685
383840,48133,276463,There s a Bat in Bunk Five,7,"providence, rhode island, usa",7.578624,7.641730,7.6,7.578624,7.642559,7.927010,7.853196,6.2,7.502718


In [12]:
len(merged_data['User-ID'].unique())

68091

In [13]:
# Выбираем 5 случайных пользователей
random_users = merged_data.sample(5)
random_users

,ISBN,User-ID,Book-Title,Book-Rating,Location,Prediction_bow_linreg,Prediction_bow_xgb,Prediction_bow_knn,Prediction_tfidf_linreg,Prediction_tfidf_xgb,Prediction_w2v_linreg,Prediction_w2v_xgb,Prediction_w2v_knn,Mean_Prediction
16602,51799,272573,Where the Heart Is Oprah s Book Club Paperba...,9,"poway, california, usa",7.579935,7.749629,7.4,7.579935,7.794161,8.068334,7.846675,7.6,7.702334
351249,86451,78744,Fun Run and Other Oxymorons Singular Reflect...,8,"sydney, new south wales, australia",7.645275,7.529050,6.8,7.645275,7.527846,7.727187,7.662128,6.4,7.367095
1793,45453,227520,Purity in Death,9,"san jose, california, usa",7.595123,7.609565,9.0,7.595123,7.694761,7.764441,7.769453,8.2,7.903558
143665,100457,11676,The Senator s Daughter,5,"n/a, n/a, n/a",7.667883,7.529050,7.6,7.667883,7.527846,8.094929,7.817770,7.0,7.613170
25144,35807,143809,Neverwhere,9,"evanston, illinois, usa",7.623341,7.605502,8.6,7.623341,7.619331,7.876150,7.727414,7.0,7.709385


In [14]:
# Оставляем только пользователей, у которых боольше 100 отзывов
new_df=merged_data[merged_data['User-ID'].map(merged_data['User-ID'].value_counts()) > 100]
new_df

,ISBN,User-ID,Book-Title,Book-Rating,Location,Prediction_bow_linreg,Prediction_bow_xgb,Prediction_bow_knn,Prediction_tfidf_linreg,Prediction_tfidf_xgb,Prediction_w2v_linreg,Prediction_w2v_xgb,Prediction_w2v_knn,Mean_Prediction
1,39,11676,Clara Callan,8,"n/a, n/a, n/a",7.667883,7.529050,8.0,7.667883,7.527846,7.992755,7.706393,7.0,7.636476
12,32114,110912,Flu The Story of the Great Influenza Pandemic...,10,"milpitas, california, usa",7.634431,7.647022,9.6,7.634431,7.675835,8.079660,7.811755,8.6,8.085392
15,32114,197659,Flu The Story of the Great Influenza Pandemic...,9,"indiana, pennsylvania, usa",7.605189,7.641730,9.2,7.605189,7.998632,8.235268,8.179282,8.8,8.158161
17,42382,11676,The Kitchen God s Wife,9,"n/a, n/a, n/a",7.667883,7.529050,8.0,7.667883,7.527846,8.102538,7.844762,9.2,7.942495
19,42382,46398,The Kitchen God s Wife,9,"san antonio, texas, usa",7.656178,7.597289,7.2,7.656178,7.616047,8.092342,7.914158,8.2,7.741524
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
383781,55425,268110,The Banished Bride Signet Regency Romance,7,"westerville, ohio, usa",7.581440,7.547065,7.6,7.581440,7.513663,7.848182,7.854055,6.6,7.515731
383821,63787,269566,The Jungle Books and Just So Stories,8,"seattle, washington, usa",7.580949,7.641730,8.2,7.580949,7.552664,8.088235,7.918825,8.2,7.845419
383822,104634,269566,Off The Wall Mad Libs No 6,6,"seattle, washington, usa",7.580949,7.641730,7.4,7.580949,7.552664,7.834584,7.762585,7.8,7.644183
383823,41773,270713,Walking With the Great Apes Jane Goodall Dia...,10,"toronto, ontario, canada",7.580562,7.529050,8.0,7.580562,7.527846,7.684317,7.755559,7.2,7.607237


In [15]:
len(new_df['User-ID'].unique())

440

In [16]:
users_pivot=new_df.pivot_table(index=["User-ID"],columns=["Book-Title"],values="Book-Rating")
users_pivot.fillna(0,inplace=True)
users_pivot.head()

Book-Title,A Nutshell handbook,AND THE HORSE HE RODE IN ON THE PEOPLE V KENNETH STARR,And Ladies of the Club,How to Travel with a Salmon and Other Essays,Lamb to the Slaughter and Other Stories Penguin 60s S,Mein verwundetes Herz Das Leben der Lilli Jahn 1900 1944,y los 40 ladrones ColecciÃ³n FÃ bula,101 Things You Can Teach Your Kids About Baseball Actually There s at Least 367 of Them,14 In The Bank Cathy Collection,A is for Alibi A Kinsey Millhone Mystery A Kinsey Millhone Mystery,...,termcap amp terminfo O Reilly Nutshell,the Dark Light Years,them Modern Library,together by christmas,wet sand raven tracks,whataboutrick com a poetic tribute to Richard A Ricci,Â Corre perro corre,Â Eres tu mi mamÃ Are You My Mother,Ã ngeles fugaces Falling Angels,Ã Â ber das Fernsehen
User-ID,,,,,,,,,,,,,,,,,,,,,
2033,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2110,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2276,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4017,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4385,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [17]:
def users_choice(id):
    users_fav_five =new_df[new_df["User-ID"]==id].sort_values(["Book-Rating"],ascending=False)[0:5]
    return users_fav_five

In [18]:
# user_id = random.choice(new_df["User-ID"].values)
user_id = 31826
user_choice_df = pd.DataFrame(users_choice(user_id))
user_favorite = users_choice(user_id)
n = len(user_choice_df["Book-Title"].values)

print("Пользователь: {} ".format(user_id))

print("\nЛюбимые книги:")
for i in range(n):
    book_title = user_choice_df["Book-Title"].tolist()[i]
    user_rating = new_df[(new_df["User-ID"] == user_id) & (new_df["Book-Title"] == book_title)]["Book-Rating"].values[0]
    predicted_rating = new_df[new_df["Book-Title"] == book_title]["Mean_Prediction"].values[0]
    print(f"Книга: {book_title} \nРейтинг пользователя: {user_rating} \nПредсказанный рейтинг моделями (средний): {predicted_rating}")


Пользователь: 31826 

Любимые книги:
Книга: Disney s Aladdin  Disney Classic Series  
Рейтинг пользователя: 10 
Предсказанный рейтинг моделями (средний): 8.815493705624831
Книга: Disney s Family Storybook Collection   75 Fables for Living  Loving  and Learning  Disneys  
Рейтинг пользователя: 10 
Предсказанный рейтинг моделями (средний): 8.761183982617759
Книга: Beatrix Potter  Artist  Stroyteller and Countrywoman 
Рейтинг пользователя: 10 
Предсказанный рейтинг моделями (средний): 8.619740430144768
Книга: A Tolkien Miscellany 
Рейтинг пользователя: 10 
Предсказанный рейтинг моделями (средний): 8.510036222854003
Книга: The Lord of the Rings  The QPB Companion to the Lord of the Rings  The Companion to the Lord of the Rings  
Рейтинг пользователя: 10 
Предсказанный рейтинг моделями (средний): 8.943980040389448


In [19]:
# Получаем рейтинги пользователя из таблицы
user_ratings = users_pivot.loc[user_id].values.reshape(1, -1)

# Рассчитываем схожесть между пользователем и другими пользователями
similarities = cosine_similarity(users_pivot, user_ratings)

# Получаем индексы пользователей с похожими предпочтениями
similar_users_index = similarities.argsort(axis=0)[-2:-7:-1].flatten()

# Получаем книги, которые понравились похожим пользователям, но которые пользователь еще не оценил
recommended_books = set()
for index in similar_users_index:
    similar_user_ratings = users_pivot.iloc[index]
    recommended_books.update(similar_user_ratings[similar_user_ratings > 0].index)

user_rated_books = set(users_pivot.columns[users_pivot.loc[user_id] > 0].values)
recommended_books -= user_rated_books

# Выводим топ-5 рекомендованных книг по убыванию предсказанного рейтинга
print("\nТоп-5 рекомендованных книг:")
for book_title in sorted(recommended_books, key=lambda x: new_df[new_df["Book-Title"] == x]["Prediction_w2v_xgb"].values[0], reverse=True)[:5]:
    predicted_rating = new_df[new_df["Book-Title"] == book_title]["Prediction_w2v_xgb"].values[0]
    print(f"Книга: {book_title} | Предсказанный рейтинг: {predicted_rating}")



Топ-5 рекомендованных книг:
Книга: House Harkonnen  Dune  House Trilogy  Book 2  | Предсказанный рейтинг: 8.764508
Книга: The Butlerian Jihad  Dune Series  | Предсказанный рейтинг: 8.633148
Книга: Shade of the Tree | Предсказанный рейтинг: 8.364219
Книга: The Winter of Our Discontent  Penguin Twentieth Century Classics  | Предсказанный рейтинг: 8.279923
Книга: Calvin and Hobbes | Предсказанный рейтинг: 8.277538


# Сравнение с добавлением возраста

In [20]:
data_w2v_xgb_age = pd.read_csv("data_w2v_xgb_age.csv")
data_w2v_xgb_age.rename(columns={'Predicted-Book-Rating': 'Prediction_w2v_xgb_age'}, inplace=True)
data_w2v_xgb_age

,ISBN,Book-Title,User-ID,Book-Rating,Location,Age,Prediction_w2v_xgb_age
0,39,Clara Callan,67544,8,"toronto, ontario, canada",30,7.773178
1,39,Clara Callan,219008,7,"halifax, nova scotia, canada",60,7.623041
2,39,Clara Callan,263325,6,"fredericton, new brunswick, canada",27,7.730576
3,4678,Decision in Normandy,2954,8,"wichita, kansas, usa",71,8.111403
4,4678,Decision in Normandy,152827,7,"ottawa, ontario, canada",40,7.757004
...,...,...,...,...,...,...,...
269616,41460,Dreamsnake,275318,10,"klamath falls, oregon, usa",49,7.581114
269617,129765,Cocktail Classics,275970,7,"pittsburgh, pennsylvania, usa",46,7.879545
269618,53784,Flashpoints Promise and Peril in a New World,276313,5,"livermore, california, usa",36,7.757675
269619,48133,There s a Bat in Bunk Five,276463,7,"providence, rhode island, usa",26,7.683237


In [21]:
# Оставляем только пользователей, у которых боольше 100 отзывов
new_df=data_w2v_xgb_age[data_w2v_xgb_age['User-ID'].map(data_w2v_xgb_age['User-ID'].value_counts()) > 100]
new_df

,ISBN,Book-Title,User-ID,Book-Rating,Location,Age,Prediction_w2v_xgb_age
6,32114,Flu The Story of the Great Influenza Pandemic...,110912,10,"milpitas, california, usa",36,7.938191
9,32114,Flu The Story of the Great Influenza Pandemic...,197659,9,"indiana, pennsylvania, usa",49,8.111135
11,42382,The Kitchen God s Wife,46398,9,"san antonio, texas, usa",37,7.924415
14,42382,The Kitchen God s Wife,230522,7,"colorado springs, colorado, usa",52,8.020420
18,45144,What If The World s Foremost Military Histor...,52614,8,"toccoa, ga., usa",33,7.921663
...,...,...,...,...,...,...,...
269560,46302,Raccoons on the Roof Animal Ark,266056,7,"hoover, alabama, usa",47,7.762393
269600,63787,The Jungle Books and Just So Stories,269566,8,"seattle, washington, usa",9,7.807955
269601,104634,Off The Wall Mad Libs No 6,269566,6,"seattle, washington, usa",9,7.689939
269602,41773,Walking With the Great Apes Jane Goodall Dia...,270713,10,"toronto, ontario, canada",27,7.754488


In [22]:
users_pivot=new_df.pivot_table(index=["User-ID"],columns=["Book-Title"],values="Book-Rating")
users_pivot.fillna(0,inplace=True)
users_pivot.head()

Book-Title,AND THE HORSE HE RODE IN ON THE PEOPLE V KENNETH STARR,And Ladies of the Club,How to Travel with a Salmon and Other Essays,Lamb to the Slaughter and Other Stories Penguin 60s S,y los 40 ladrones ColecciÃ³n FÃ bula,101 Things You Can Teach Your Kids About Baseball Actually There s at Least 367 of Them,14 In The Bank Cathy Collection,A is for Alibi A Kinsey Millhone Mystery A Kinsey Millhone Mystery,Any Grooming Hints for Your Fans Rollie A Doonesbury book,Arriba Comunicacion y cultura,...,flambeau darkcorp com,good girls do Sex Chronicles of a Shameless Generation,one hundred years of solitude,operation mistletoe the special agents,sharks reading discovery,stardust,the Dark Light Years,together by christmas,Â Eres tu mi mamÃ Are You My Mother,Ã ngeles fugaces Falling Angels
User-ID,,,,,,,,,,,,,,,,,,,,,
2033,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2276,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4017,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4385,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5582,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [23]:
# user_id = random.choice(new_df["User-ID"].values)
user_id = 31826
user_choice_df = pd.DataFrame(users_choice(user_id))
user_favorite = users_choice(user_id)
n = len(user_choice_df["Book-Title"].values)

print("Пользователь: {} ".format(user_id))

print("\nЛюбимые книги:")
for i in range(n):
    book_title = user_choice_df["Book-Title"].tolist()[i]
    user_rating = new_df[(new_df["User-ID"] == user_id) & (new_df["Book-Title"] == book_title)]["Book-Rating"].values[0]
    predicted_rating = new_df[new_df["Book-Title"] == book_title]["Prediction_w2v_xgb_age"].values[0]
    print(f"Книга: {book_title} \nРейтинг пользователя: {user_rating} \nПредсказанный рейтинг моделями (средний): {predicted_rating}")


Пользователь: 31826 

Любимые книги:
Книга: Disney s Aladdin  Disney Classic Series  
Рейтинг пользователя: 10 
Предсказанный рейтинг моделями (средний): 8.371922
Книга: Disney s Family Storybook Collection   75 Fables for Living  Loving  and Learning  Disneys  
Рейтинг пользователя: 10 
Предсказанный рейтинг моделями (средний): 8.31444
Книга: Beatrix Potter  Artist  Stroyteller and Countrywoman 
Рейтинг пользователя: 10 
Предсказанный рейтинг моделями (средний): 8.368956
Книга: A Tolkien Miscellany 
Рейтинг пользователя: 10 
Предсказанный рейтинг моделями (средний): 8.263604
Книга: The Lord of the Rings  The QPB Companion to the Lord of the Rings  The Companion to the Lord of the Rings  
Рейтинг пользователя: 10 
Предсказанный рейтинг моделями (средний): 8.4013605


In [24]:
# Получаем рейтинги пользователя из таблицы
user_ratings = users_pivot.loc[user_id].values.reshape(1, -1)

# Рассчитываем схожесть между пользователем и другими пользователями
similarities = cosine_similarity(users_pivot, user_ratings)

# Получаем индексы пользователей с похожими предпочтениями
similar_users_index = similarities.argsort(axis=0)[-2:-7:-1].flatten()

# Получаем книги, которые понравились похожим пользователям, но которые пользователь еще не оценил
recommended_books = set()
for index in similar_users_index:
    similar_user_ratings = users_pivot.iloc[index]
    recommended_books.update(similar_user_ratings[similar_user_ratings > 0].index)

user_rated_books = set(users_pivot.columns[users_pivot.loc[user_id] > 0].values)
recommended_books -= user_rated_books

# Выводим топ-5 рекомендованных книг по убыванию предсказанного рейтинга
print("\nТоп-5 рекомендованных книг:")
for book_title in sorted(recommended_books, key=lambda x: new_df[new_df["Book-Title"] == x]["Prediction_w2v_xgb_age"].values[0], reverse=True)[:5]:
    predicted_rating = new_df[new_df["Book-Title"] == book_title]["Prediction_w2v_xgb_age"].values[0]
    print(f"Книга: {book_title} | Предсказанный рейтинг: {predicted_rating}")



Топ-5 рекомендованных книг:
Книга: It s A Magical World  A Calvin and Hobbes Collection | Предсказанный рейтинг: 8.306374
Книга: Harry Potter and the Goblet of Fire  Book 4  | Предсказанный рейтинг: 8.300982
Книга: The Best of H  P  Lovecraft  Bloodcurdling Tales of Horror and the Macabre | Предсказанный рейтинг: 8.298372
Книга: The Renegades of Pern  Dragonriders of Pern  Paperback   | Предсказанный рейтинг: 8.289438
Книга: Singers of Strange Songs  A Celebration of Brian Lumley  Call of Cthulhu Fiction  | Предсказанный рейтинг: 8.286798
